In [9]:
import requests
import pandas as pd
import folium
import plotly.express as px
MAPBOX_API_KEY = 'pk.eyJ1IjoibW9ua3oxMSIsImEiOiJja3ZjcGpkcmNhemljMnBuemllb294Z20yIn0.LFHcV2_PcE_wc5ttNnMM1g'
px.set_mapbox_access_token(MAPBOX_API_KEY)

In [10]:
categories_fr = ['Introduction','Vol dans / sur véhicule à moteur','Vol de véhicule à moteur','Méfait','Vol qualifié','Infraction entraînant la mort']
categories_en = ['Breaking and entering','Theft from a vehicle/theft of vehicle parts','Vehicle theft','General damages','Theft with violence','Murder']
categories_fr_to_en = dict(zip(categories_fr,categories_en))

times_of_day_fr = ['jour','soir','nuit']
times_of_day_en = ['Day','Evening','Night']
times_of_day_fr_to_en = dict(zip(times_of_day_fr,times_of_day_en))



In [11]:
def get_df(year,limit='100000'):
    url = 'https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=c6f482bf-bf0f-4960-8b2f-9982c211addd'
    params = {'q':f'{year}-', 'limit':limit}
    response = requests.get(url,params=params)
    d = response.json()
    df = pd.DataFrame(d['result']['records'])
    return df

In [12]:
def filter_df(df, categories, times_of_day):
    df = df[df['CATEGORIE'].isin(categories)]
    df = df[df['QUART'].isin(times_of_day)]
    return df

In [20]:
def preprocess_df(df):
    df["LONGITUDE"] = pd.to_numeric(df['LONGITUDE'], errors="coerce")
    df["LATITUDE"] = pd.to_numeric(df['LATITUDE'], errors="coerce")
    df["CATEGORIE"] = df["CATEGORIE"].map(categories_fr_to_en)
    df["QUART"] = df["QUART"].map(times_of_day_fr_to_en)

    return df.dropna()

In [14]:
def get_map(df):
    fig = px.scatter_mapbox(df, lat="LATITUDE", lon="LONGITUDE",  color="CATEGORIE",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=5, zoom=10)
    return fig

In [53]:
def generate_map(year,categories,times_of_day,limit='100000'):
    df = get_df(year,limit)
    df = preprocess_df(df)
    df = filter_df(df,categories,times_of_day)
    location_row = {'_id':'0','CATEGORIE':'Starting location','QUART':'null','PDQ':'null','X':'null','Y':'null','LATITUDE':45.6,'LONGITUDE':-73.5,'rank':'null'}
    df = df._append(location_row, ignore_index=True)
    my_map = get_map(df)
    return my_map

In [21]:
categories = ['Introduction','Vol dans / sur véhicule à moteur','Vol de véhicule à moteur','Méfait','Vol qualifié','Infraction entraînant la mort']
times_of_day = ['jour','soir','nuit']
df = get_df(2024)
df = preprocess_df(df)
df

,_id,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE,rank
0,162778,Breaking and entering,2024-01-13,Night,20,299244.49100599,5040219.74599321,-73.571090,45.501877,0.057309
1,162782,Breaking and entering,2024-01-15,Night,7,287492.598989491,5038225.1219981,-73.721399,45.483736,0.057309
2,162783,Breaking and entering,2024-01-15,Day,27,291643.014000033,5044760.53699814,-73.668482,45.542635,0.057309
3,162784,Breaking and entering,2024-01-17,Day,31,294347.117997179,5043159.06499251,-73.633821,45.528270,0.057309
4,162793,Breaking and entering,2024-01-19,Day,20,299244.49100599,5040219.74599321,-73.571090,45.501877,0.057309
...,...,...,...,...,...,...,...,...,...,...
1785,278831,Breaking and entering,2024-01-14,Evening,22,300466.906997352,5042151.08499503,-73.555465,45.519265,0.057309
1786,278832,Breaking and entering,2024-01-16,Night,23,302178.378996452,5045740.86799442,-73.533577,45.551576,0.057309
1787,278833,Breaking and entering,2024-01-19,Day,21,300596.273991037,5040627.6990001,-73.553796,45.505558,0.057309
1788,278834,Breaking and entering,2024-01-19,Evening,15,298258.986002694,5036961.03799534,-73.583657,45.472545,0.057309


In [31]:
categories_en = ['Breaking and entering','Theft from a vehicle/theft of vehicle parts','Vehicle theft','General damages','Theft with violence','Murder']
times_of_day_en = ['Day','Evening','Night']
fig = generate_map(2024,categories=categories_en,times_of_day=times_of_day_en)

/home/jikael/anaconda3/envs/mchacks/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [57]:
my_map = generate_map(year=2024,
                      categories=categories_en,
                      times_of_day = times_of_day_en,)
my_map.update_traces(
    selector=dict(mode='markers', marker=dict(CATEGORIE='Starting location')),
    marker=dict(size=14, color='red', symbol='star')
)

/home/jikael/anaconda3/envs/mchacks/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [62]:
new_point_data = dict()
my_map.update_traces(new_point_data)

In [66]:
existing_map = px.scatter_mapbox(
    lat=[40.7128, 41.8781],  # Existing latitude data
    lon=[-74.0060, -87.6298],  # Existing longitude data
    text=['Point A', 'Point B'],  # Labels for existing points
    zoom=10
)
existing_map